In [ ]:
!pip install pandas numpy sentence-transformers faiss-cpu scikit-learn  groq
# For Groq, follow their specific documentation or API guide.
!pip install langchain sentence-transformers faiss-cpu pandas openai
!pip install langchain-community
!pip install langchain-community
!pip install --upgrade langchain langchain-community
!pip install PyPDF2
!pip install pymupdf
!pip install transformers # Install the missing langchain-community package# Install the missing langchain-community package

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

Semantic chunking

In [ ]:
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAIChat
from langchain.prompts import ChatPromptTemplate
from groq import Groq

# Load your dataset from Excel
file_path = '/content/Data.xlsx'
df = pd.read_excel(file_path)

# Trim whitespace from the column names
df.columns = df.columns.str.strip()

# Extract problems and solutions
try:
    problems = df['Problem'].tolist()
    solutions = df['Solution'].tolist()
except KeyError as e:
    print(f"KeyError: {e} - Please check the column names.")
    exit()  # Exit if there is a KeyError

# Load a pre-trained sentence transformer model for embedding the problems
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings_model = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')  # Corrected here

# Create a DataFrameLoader - Remove the data argument
# The text_column argument is available in newer versions of LangChain
loader = DataFrameLoader(df, page_content_column='Problem') # Use page_content_column instead of text_column

# Split documents into chunks
splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10)

# Load documents and then split them using the splitter
documents = loader.load()
documents = splitter.split_documents(documents)

# Create FAISS vector store
vector_store = FAISS.from_documents(documents, embeddings_model)

# Initialize Groq client
client = Groq(api_key="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")  # Replace with your actual API key

# Function to use Groq API to refine the query
def refine_query_with_groq(query_text):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI assistant that refines user queries to improve search accuracy. Provide a concise, refined version of the user's query.",
                },
                {
                    "role": "user",
                    "content": f"Refine the following problem description for better search accuracy: {query_text}",
                }
            ],
            model="llama3-8b-8192",
            max_tokens=100,
        )
        return chat_completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error during query refinement: {e}")
        return query_text  # Return the original query in case of error

# Example query to test the index
query_text = input("Enter your query or problem: ")

# Use Groq to refine the query
refined_query_text = refine_query_with_groq(query_text)
print(f"Refined query: {refined_query_text}")

# Perform the search for the top k most similar items
retrieved_documents = vector_store.similarity_search(refined_query_text, k=5)

# Combine retrieved documents as context for the generative model
retrieval_context = ""
for doc in retrieved_documents:
    retrieval_context += f"Problem Chunk: {doc.page_content}\n"



# Function to use Groq API and combine retrieval and generation (RAG)
def generate_answer_with_rag(refined_query_text, retrieval_context):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI assistant that generates answers to user queries based on retrieved context.",
                },
                {
                    "role": "user",
                    "content": f"Using the following retrieved context, provide a detailed solution to the query: {retrieval_context}\nQuery: {refined_query_text}",
                }
            ],
            model="llama3-8b-8192",  # Assuming you're using LLaMA 3
            max_tokens=500,
        )
        return chat_completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error during answer generation: {e}")
        return "Sorry, I couldn't generate an answer."  # Return a default message in case of error

# Generate the final answer with RAG
final_answer = generate_answer_with_rag(refined_query_text, retrieval_context)
print(f"Final Answer:\n{final_answer}")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-4-7a8915777441>:31: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')  # Corrected here


Enter your query or problem: network router is not configurating 
Refined query: Here is a refined version of the query for better search accuracy:

"Network router configuration not working or not applying changes"

I refined the query by:

* Breaking the phrase "not configurating" into two words "not" and "configuring" to make the query more specific and searchable.
* Changing "is not" to "not" to make the query more concise.
* Adding the phrase "not applying changes" to make the query more specific about the issue, increasing the chances of finding
Final Answer:
Based on the retrieved context, I'd be happy to help you troubleshoot the issue with your network router configuration not working or not applying changes.

Given the complexity of network configurations, it's essential to methodically approach the problem to identify the root cause. Here's a step-by-step approach to help you resolve the issue:

1. **Check routing tables**: Verify that the routing tables on the router are co

Fixed chunking

In [ ]:
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAIChat
from langchain.prompts import ChatPromptTemplate
from groq import Groq

# Load your dataset from Excel
file_path = '/content/Data.xlsx'
df = pd.read_excel(file_path)

# Trim whitespace from the column names
df.columns = df.columns.str.strip()

# Extract problems and solutions
try:
    problems = df['Problem'].tolist()
    solutions = df['Solution'].tolist()
except KeyError as e:
    print(f"KeyError: {e} - Please check the column names.")
    exit()  # Exit if there is a KeyError

# Load a pre-trained sentence transformer model for embedding the problems
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings_model = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')  # Corrected here

# Create a DataFrameLoader
loader = DataFrameLoader(df, page_content_column='Problem')  # Use page_content_column instead of text_column

# Split documents into fixed-size chunks
chunk_size = 100  # Set your fixed chunk size
splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)  # Set overlap to 0 for fixed chunking

# Load documents and then split them using the splitter
documents = loader.load()
documents = splitter.split_documents(documents)

# Create FAISS vector store
vector_store = FAISS.from_documents(documents, embeddings_model)

# Initialize Groq client
client = Groq(api_key="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")  # Replace with your actual API key

# Function to use Groq API to refine the query
def refine_query_with_groq(query_text):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI assistant that refines user queries to improve search accuracy. Provide a concise, refined version of the user's query.",
                },
                {
                    "role": "user",
                    "content": f"Refine the following problem description for better search accuracy: {query_text}",
                }
            ],
            model="llama3-8b-8192",
            max_tokens=100,
        )
        return chat_completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error during query refinement: {e}")
        return query_text  # Return the original query in case of error

# Example query to test the index
query_text = input("Enter your query or problem: ")

# Use Groq to refine the query
refined_query_text = refine_query_with_groq(query_text)
print(f"Refined query: {refined_query_text}")

# Perform the search for the top k most similar items
retrieved_documents = vector_store.similarity_search(refined_query_text, k=5)

# Combine retrieved documents as context for the generative model
retrieval_context = ""
for doc in retrieved_documents:
    retrieval_context += f"Problem Chunk: {doc.page_content}\n"

# Function to use Groq API and combine retrieval and generation (RAG)
def generate_answer_with_rag(refined_query_text, retrieval_context):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI assistant that generates answers to user queries based on retrieved context.",
                },
                {
                    "role": "user",
                    "content": f"Using the following retrieved context, provide a detailed solution to the query: {retrieval_context}\nQuery: {refined_query_text}",
                }
            ],
            model="llama3-8b-8192",  # Assuming you're using LLaMA 3
            max_tokens=500,
        )
        return chat_completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error during answer generation: {e}")
        return "Sorry, I couldn't generate an answer."  # Return a default message in case of error

# Generate the final answer with RAG
final_answer = generate_answer_with_rag(refined_query_text, retrieval_context)
print(f"Final Answer:\n{final_answer}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Enter your query or problem: network router is not configurating 
Refined query: Here's a refined version of the query:

"Troubleshooting network router configuration issues"

This refined query is more concise, specific, and accurate. I replaced "not configurating" with "configuration issues" to better convey the problem, and added "troubleshooting" to indicate that you're looking for solutions or advice on how to resolve the issue. This should help improve search accuracy and yield more relevant results.
Final Answer:
Based on the retrieved context, I'll provide a detailed solution to troubleshoot network router configuration issues.

**Step 1: Verify Network Connectivity**

Before diving into router configuration, ensure that your network devices are connected properly and functioning correctly. Check the following:

* Physical connections: Ensure that all cables are securely plugged in and free from damage.
* Power cycles: Restart devices such as routers, switches, and modems to ru

Recursive chunking

In [ ]:
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAIChat
from langchain.prompts import ChatPromptTemplate
from groq import Groq

# Load your dataset from Excel
file_path = '/content/Data.xlsx'
df = pd.read_excel(file_path)

# Trim whitespace from the column names
df.columns = df.columns.str.strip()

# Extract problems and solutions
try:
    problems = df['Problem'].tolist()
    solutions = df['Solution'].tolist()
except KeyError as e:
    print(f"KeyError: {e} - Please check the column names.")
    exit()  # Exit if there is a KeyError

# Load a pre-trained sentence transformer model for embedding the problems
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings_model = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')  # Corrected here

# Create a DataFrameLoader
loader = DataFrameLoader(df, page_content_column='Problem')  # Use page_content_column instead of text_column

# Initialize RecursiveCharacterTextSplitter with desired chunk size and overlap
max_chunk_size = 200  # Maximum size of each chunk
# min_chunk_size = 100  # Minimum size of chunks - Removed this line
splitter = RecursiveCharacterTextSplitter(
    # chunk_size=max_chunk_size,  - This line was modified
    chunk_size = min(max_chunk_size, 1000), # chunk_size has a limit of 1000
    chunk_overlap=20,
    # min_chunk_size=min_chunk_size, - This line was removed
    length_function=len
)

# Load documents and then split them using the splitter
documents = loader.load()
documents = splitter.split_documents(documents)

# Create FAISS vector store
vector_store = FAISS.from_documents(documents, embeddings_model)

# Initialize Groq client
client = Groq(api_key="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")  # Replace with your actual API key

# Function to use Groq API to refine the query
def refine_query_with_groq(query_text):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI assistant that refines user queries to improve search accuracy. Provide a concise, refined version of the user's query.",
                },
                {
                    "role": "user",
                    "content": f"Refine the following problem description for better search accuracy: {query_text}",
                }
            ],
            model="llama3-8b-8192",
            max_tokens=100,
        )
        return chat_completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error during query refinement: {e}")
        return query_text  # Return the original query in case of error

# Example query to test the index
query_text = input("Enter your query or problem: ")

# Use Groq to refine the query
refined_query_text = refine_query_with_groq(query_text)
print(f"Refined query: {refined_query_text}")

# Perform the search for the top k most similar items
retrieved_documents = vector_store.similarity_search(refined_query_text, k=5)

# Combine retrieved documents as context for the generative model
retrieval_context = ""
for doc in retrieved_documents:
    retrieval_context += f"Problem Chunk: {doc.page_content}\n"

# Function to use Groq API and combine retrieval and generation (RAG)
def generate_answer_with_rag(refined_query_text, retrieval_context):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI assistant that generates answers to user queries based on retrieved context.",
                },
                {
                    "role": "user",
                    "content": f"Using the following retrieved context, provide a detailed solution to the query: {retrieval_context}\nQuery: {refined_query_text}",
                }
            ],
            model="llama3-8b-8192",  # Assuming you're using LLaMA 3
            max_tokens=500,
        )
        return chat_completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error during answer generation: {e}")
        return "Sorry, I couldn't generate an answer."  # Return a default message in case of error

# Generate the final answer with RAG
final_answer = generate_answer_with_rag(refined_query_text, retrieval_context)
print(f"Final Answer:\n{final_answer}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Enter your query or problem: network router is not configurating 
Refined query: Here's a refined version of the query:

"Router configuration issues: unable to configure or connect to a network router"

This refined query includes:

* Specific keywords: "Router configuration" and "issues"
* Clarified intent: The user is seeking help with configuring a network router, which is not functioning as expected
* Additional context: "unable to configure or connect to a network router" provides more details about the issue and its scope
Final Answer:
Based on the retrieved context and the refined query, I'll provide a detailed solution to help the user resolve the issue with configuring or connecting to a network router.

**Possible Causes:**

Based on the context, the issue could be caused by one or a combination of the following factors:

1. Misconfigured Routing Tables: The router's routing tables might be misconfigured, preventing the router from properly communicating with other devices o

Documentation chunking

In [ ]:
!pip install langchain>=0.0.260

In [ ]:
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAIChat
from langchain.prompts import ChatPromptTemplate
from groq import Groq

# Load your dataset from Excel
file_path = '/content/Data.xlsx'
df = pd.read_excel(file_path)

# Trim whitespace from the column names
df.columns = df.columns.str.strip()

# Extract problems and solutions
try:
    problems = df['Problem'].tolist()
    solutions = df['Solution'].tolist()
except KeyError as e:
    print(f"KeyError: {e} - Please check the column names.")
    exit()  # Exit if there is a KeyError

# Load a pre-trained sentence transformer model for embedding the problems
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings_model = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')  # Corrected here

# Create a DataFrameLoader
loader = DataFrameLoader(df, page_content_column='Problem')  # Use page_content_column instead of text_column

# Function to perform documentation chunking based on paragraph breaks
def documentation_chunking(documents):
    chunks = []
    for doc in documents:
        # Split document by newlines (paragraph breaks)
        paragraphs = doc.page_content.split('\n\n')  # Assuming double newlines separate paragraphs
        for paragraph in paragraphs:
            if paragraph.strip():  # Ignore empty paragraphs
                chunks.append(paragraph.strip())
    return chunks

# Load documents
documents = loader.load()
# Apply documentation chunking
document_chunks = documentation_chunking(documents)

# Create FAISS vector store
vector_store = FAISS.from_documents(documents, embeddings_model)
# Initialize Groq client
client = Groq(api_key="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")  # Replace with your actual API key

# Function to use Groq API to refine the query
def refine_query_with_groq(query_text):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI assistant that refines user queries to improve search accuracy. Provide a concise, refined version of the user's query.",
                },
                {
                    "role": "user",
                    "content": f"Refine the following problem description for better search accuracy: {query_text}",
                }
            ],
            model="llama3-8b-8192",
            max_tokens=100,
        )
        return chat_completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error during query refinement: {e}")
        return query_text  # Return the original query in case of error

# Example query to test the index
query_text = input("Enter your query or problem: ")

# Use Groq to refine the query
refined_query_text = refine_query_with_groq(query_text)
print(f"Refined query: {refined_query_text}")

# Perform the search for the top k most similar items
retrieved_documents = vector_store.similarity_search(refined_query_text, k=5)

# Combine retrieved documents as context for the generative model
retrieval_context = ""
for doc in retrieved_documents:
    retrieval_context += f"Problem Chunk: {doc}\n"  # Change this to handle string chunks

# Function to use Groq API and combine retrieval and generation (RAG)
def generate_answer_with_rag(refined_query_text, retrieval_context):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI assistant that generates answers to user queries based on retrieved context.",
                },
                {
                    "role": "user",
                    "content": f"Using the following retrieved context, provide a detailed solution to the query: {retrieval_context}\nQuery: {refined_query_text}",
                }
            ],
            model="llama3-8b-8192",  # Assuming you're using LLaMA 3
            max_tokens=500,
        )
        return chat_completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error during answer generation: {e}")
        return "Sorry, I couldn't generate an answer."  # Return a default message in case of error

# Generate the final answer with RAG
final_answer = generate_answer_with_rag(refined_query_text, retrieval_context)
print(f"Final Answer:\n{final_answer}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Enter your query or problem: network router is not configurating 
Refined query: Refined Query: 

"Router configuration not working or not saving, troubleshooting and solutions"
Final Answer:
Based on the retrieved context, there are several possible causes for router configuration not working or not saving. Here are some potential solutions and troubleshooting steps you can follow:

1. **Malfunctioning Routers or Switches**: If the router is malfunctioning or faulty, it may be causing issues with the configuration not saving. Try restarting or resetting the router to its default settings. If the issue persists, it may be necessary to replace the router or switch.

   Key points to check:
   - Reboot or reset devices
   - Check connections

2. **DNS Configuration Issues**: If the DNS configuration is incorrect, it may cause issues with the router configuration not saving. Verify that the DNS server settings are correct, and ensure that the forwarders and root hints are properly configu

Agentic chunking

In [ ]:
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAIChat
from langchain.prompts import ChatPromptTemplate
from groq import Groq
from langchain.schema import Document

# Load your dataset from Excel
file_path = '/content/Data.xlsx'
df = pd.read_excel(file_path)

# Trim whitespace from the column names
df.columns = df.columns.str.strip()

# Extract problems and solutions
try:
    problems = df['Problem'].tolist()
    solutions = df['Solution'].tolist()
except KeyError as e:
    print(f"KeyError: {e} - Please check the column names.")
    exit()  # Exit if there is a KeyError

# Load a pre-trained sentence transformer model for embedding the problems
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings_model = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')  # Corrected here

# Create a DataFrameLoader
loader = DataFrameLoader(df, page_content_column='Problem')  # Use page_content_column instead of text_column

# Function to perform agentic chunking
def agentic_chunking(documents, chunk_size=300):
    chunks = []
    current_chunk = ""

    for doc in documents:
        for paragraph in doc.page_content.split('\n\n'):  # Split by double newlines for paragraphs
            paragraph = paragraph.strip()
            if not paragraph:  # Skip empty paragraphs
                continue

            # Check if adding the paragraph exceeds the chunk size
            if len(current_chunk) + len(paragraph) + 1 <= chunk_size:
                current_chunk += f"{paragraph}\n\n"  # Add paragraph to the current chunk
            else:
                if current_chunk:  # If there's a current chunk, save it
                    chunks.append(current_chunk.strip())
                current_chunk = f"{paragraph}\n\n"  # Start a new chunk with the current paragraph

    # Add any remaining chunk
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

# Load documents
documents = loader.load()
# Apply agentic chunking
document_chunks = agentic_chunking(documents)

# Convert string chunks to Document objects
document_chunks = [Document(page_content=chunk, metadata={}) for chunk in document_chunks]

# Create FAISS vector store
vector_store = FAISS.from_documents(document_chunks, embeddings_model)
# Initialize Groq client
client = Groq(api_key="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")  # Replace with your actual API key

# Function to use Groq API to refine the query
def refine_query_with_groq(query_text):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI assistant that refines user queries to improve search accuracy. Provide a concise, refined version of the user's query.",
                },
                {
                    "role": "user",
                    "content": f"Refine the following problem description for better search accuracy: {query_text}",
                }
            ],
            model="llama3-8b-8192",
            max_tokens=100,
        )
        return chat_completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error during query refinement: {e}")
        return query_text  # Return the original query in case of error

# Example query to test the index
query_text = input("Enter your query or problem: ")

# Use Groq to refine the query
refined_query_text = refine_query_with_groq(query_text)
print(f"Refined query: {refined_query_text}")

# Perform the search for the top k most similar items
retrieved_documents = vector_store.similarity_search(refined_query_text, k=5)

# Combine retrieved documents as context for the generative model
retrieval_context = ""
for doc in retrieved_documents:
    retrieval_context += f"Problem Chunk: {doc}\n"  # Change this to handle string chunks

# Function to use Groq API and combine retrieval and generation (RAG)
def generate_answer_with_rag(refined_query_text, retrieval_context):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI assistant that generates answers to user queries based on retrieved context.",
                },
                {
                    "role": "user",
                    "content": f"Using the following retrieved context, provide a detailed solution to the query: {retrieval_context}\nQuery: {refined_query_text}",
                }
            ],
            model="llama3-8b-8192",  # Assuming you're using LLaMA 3
            max_tokens=500,
        )
        return chat_completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error during answer generation: {e}")
        return "Sorry, I couldn't generate an answer."  # Return a default message in case of error

# Generate the final answer with RAG
final_answer = generate_answer_with_rag(refined_query_text, retrieval_context)
print(f"Final Answer:\n{final_answer}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Enter your query or problem: network router is not configurating 
Refined query: Here is a refined version of your query:

"Router configuration issue: Device not configuring properly or not connecting to network"

This refined query is more specific and accurate, making it easier for search engines to return relevant results.
Final Answer:
Based on the retrieved context, the query "Router configuration issue: Device not configuring properly or not connecting to network" can be matched with potential causes from all the provided problem chunks. Here's a detailed solution:

From the provided problem chunks, the following potential causes are related to the query:

* From 'Device Connectivity Issues', possible causes include:
	+ Outdated Firmware
	+ Limited Network Range
	+ Authentication Failures
	+ Network Driver Issues
* From 'Routing Protocol Conflicts', potential causes include:
	+ Unpatched Network Device Vulnerabilities
	+ Poor Network Documentation
	+ Unmonitored Network Traffic
